<a href="https://colab.research.google.com/github/alexlaurence/DoReMi/blob/master/DoReMi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation

In [0]:
!pip install feedparser
!pip install google-cloud-texttospeech

In [0]:
import feedparser
import re, os, glob, shutil

from google.cloud import texttospeech
from google.oauth2 import service_account

from IPython.display import Audio
from scipy.io import wavfile

To generate speech, you need a credentials json file tied your Google Cloud account:

In [0]:
#set API Credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]='/content/My First Project-a8dd55fddd5e.json'

## DoReMi (v0.1.0-alpha)

![alt text](https://i.imgur.com/OS76n8F.png)

By Alexander Laurence (alexander.adamlaurence@gmail.com)

http://celestial.tokyo/~AlexLaurence/

DoReMi is a tool for blind users to search job listings from by synthesising job listing RSS feeds from Scout.co.kr into speech with the help of Google's Text-to-Speech API powered by a WaveNet model.

**Why Google Text-to-Speech?**

The Text-to-Speech API creates raw audio data of natural, human speech. That is, it creates audio that sounds like a person talking. When you send a synthesis request to the Text-to-Speech API, you must specify a voice that 'speaks' the words.

**Why WaveNet?**

A WaveNet generates speech that sounds more natural than other text-to-speech systems. It synthesizes speech with more human-like emphasis and inflection on syllables, phonemes, and words. On average, a WaveNet produces speech audio that people prefer over other text-to-speech technologies.

Unlike most other text-to-speech systems, a WaveNet model creates raw audio waveforms from scratch. The model uses a neural network that has been trained using a large volume of speech samples. During training, the network extracts the underlying structure of the speech, such as which tones follow each other and what a realistic speech waveform looks like. When given a text input, the trained WaveNet model can generate the corresponding speech waveforms from scratch, one sample at a time, with up to 24,000 samples per second and seamless transitions between the individual sounds.

In [13]:
#@title DoReMi v0.1.0-alpha

def start():
  
  credentials = service_account.Credentials.from_service_account_file('/content/My First Project-a8dd55fddd5e.json')
  client = texttospeech.TextToSpeechClient(credentials=credentials)
  
  # set xml url as strings
  seoul = "http://feeds.feedburner.com/co/clsY"
  gangwon = "http://feeds.feedburner.com/co/Utav"
  gyeonggi = "http://feeds.feedburner.com/co/lUKP"
  gyeongnam = "http://feeds.feedburner.com/co/rMCwN"
  gyeongbuk = "http://feeds.feedburner.com/co/Dhwm"
  gwangju = "http://feeds.feedburner.com/co/LXwe"
  daegu = "http://feeds.feedburner.com/co/Bcsg"
  daejeon = "http://feeds.feedburner.com/co/zBIF"
  busan = "http://feeds.feedburner.com/co/dkiFn"
  ulsan = "http://feeds.feedburner.com/co/LFJz"
  incheon = "http://feeds.feedburner.com/co/EkCu"
  jeonnam = "http://feeds.feedburner.com/co/wjlNw"
  jeonbuk = "http://feeds.feedburner.com/co/ZaXx"
  jeju = "http://feeds.feedburner.com/co/viTpC"
  chungnam = "http://feeds.feedburner.com/co/yIBb"
  chungbuk = "http://feeds.feedburner.com/co/aXuFR"
  national = "http://feeds.feedburner.com/co/TmxG"

  # ask user to enter the city
  get_city = input('도시 이름: ').lower()
  print('OK! {}. 잠시만 기다려주세요...'.format(get_city))
  
  # set the value to the corresponding xml url
  if get_city == '서울' or get_city == 'seoul':
    get_city = seoul
  elif get_city == '강원' or get_city == 'gangwon':
    get_city = gangwon
  elif get_city == '경기' or get_city == 'gyeonggi':
    get_city = gyeonggi
  elif get_city == '경남' or get_city == 'gyeongnam':
    get_city = gyeongnam
  elif get_city == '경북' or get_city == 'gyeongbuk':
    get_city = gyeongbuk
  elif get_city == '광주' or get_city == 'gwangju':
    get_city = gwangju
  elif get_city == '대구' or get_city == 'daegu':
    get_city = daegu
  elif get_city == '대전' or get_city == 'daejeon':
    get_city = daejeon
  elif get_city == '부산' or get_city == 'busan':
    get_city = busan
  elif get_city == '울산' or get_city == 'ulsan':
    get_city = ulsan
  elif get_city == '인천' or get_city == 'incheon':
    get_city = incheon
  elif get_city == '전남' or get_city == 'jeonnam':
    get_city = jeonnam
  elif get_city == '전북' or get_city == 'jeonbuk':
    get_city = jeonbuk
  elif get_city == '제주' or get_city == 'jeju':
    get_city = jeju
  elif get_city == '충남' or get_city == 'chungnam':
    get_city = chungnam
  elif get_city == '충북' or get_city == 'chungbuk':
    get_city = chungbuk
  elif get_city == '전국' or get_city == 'national':
    get_city = national
  else:
    print('미안 난 이해를 못 했어...')
    start()

  # parse RSS feed
  feed = feedparser.parse(get_city)
  feed_title = feed['feed']['title']
  feed_entries = feed.entries
  
  # set voice settings
  voice = texttospeech.types.VoiceSelectionParams(
      language_code='ko-KR',
      name='ko-KR-Wavenet-A',
      ssml_gender=texttospeech.enums.SsmlVoiceGender.FEMALE)

  audio_config = texttospeech.types.AudioConfig(
      audio_encoding=texttospeech.enums.AudioEncoding.MP3,
      speaking_rate=0.6)
  
  # Create directory
  dst = '/content/audio/'
  if not os.path.exists(dst):
    os.makedirs(dst)

  # read entries
  i=1
  for entry in feed.entries:
    article_title = entry.title
    article_description = entry.description
    article_link = entry.link
    
    clean_title = re.sub("-", "", article_title)
    clean_description = re.sub("<.*?>", "", article_description)
    clean_description2 = re.sub(":", "", clean_description)
    clean_description3 = re.sub("·", " ", clean_description2)
    clean_description4 = re.sub("-", "", clean_description3)
    
    print("\n {} {}".format(clean_title, clean_description4))
    
    # synthesise text to speech
    input_text = texttospeech.types.SynthesisInput(text=clean_title+clean_description4)
    response = client.synthesize_speech(input_text, voice, audio_config)
    
    # The response's audio_content is binary.
    with open('listing_{}.mp3'.format(i), 'wb') as out:
        out.write(response.audio_content)
        print('\n')
        display(Audio('listing_{}.mp3'.format(i),rate=44100))
        
        src = "/content/"
        shutil.move(os.path.join(src, "listing_{}.mp3".format(i)), os.path.join(dst, "listing_{}.mp3".format(i)))
        #print('Audio content written to file "output.mp3"')
                
    i=i+1
    
print('DoReMi에서 오신 것을 환영합니다!')
start()

DoReMi에서 오신 것을 환영합니다!
도시 이름: seoul
OK! seoul. 잠시만 기다려주세요...

 롯데지에프알  티렌 영업MD 단기 아르바이트 모집  회사명  롯데지에프알 모집내용  롯데지에프알  티렌 영업MD 단기 아르바이트 모집 직종   근무형태  기타 경력  경력 학력  전졸(예) 지역  서울 마감일  06/21(금) 입사지원하기 롯데지에프알 기업의 진행중인 채용정보 보기





 주)진태옥  영업 MD/영업부 책임자 모집(과장부장급)  회사명  주)진태옥 모집내용  주)진태옥  영업 MD/영업부 책임자 모집(과장부장급) 직종   근무형태  정규직 경력  경력 학력  전졸(예) 지역  서울 마감일  06/21(금) 입사지원하기 주)진태옥 기업의 진행중인 채용정보 보기





 더휴컴퍼니  영업관리 신입 및 경력자 모집  회사명  더휴컴퍼니 모집내용  더휴컴퍼니  영업관리 신입 및 경력자 모집 직종   근무형태  정규직 경력  무관 학력  전졸(예) 지역  서울 마감일  06/22(토) 입사지원하기 더휴컴퍼니 기업의 진행중인 채용정보 보기





 미쥬에프앤에프  여성의류 디자인실 신입 디자이너 채용  회사명  미쥬에프앤에프 모집내용  미쥬에프앤에프  여성의류 디자인실 신입 디자이너 채용 직종   근무형태  계약직, 인턴 경력  경력 학력  전졸(예) 지역  서울 마감일  채용시까지 입사지원하기 미쥬에프앤에프 기업의 진행중인 채용정보 보기





 렙쇼메이  르피타 디자인실 보조/피팅 모집  회사명  렙쇼메이 모집내용  렙쇼메이  르피타 디자인실 보조/피팅 모집 직종   근무형태  계약직 경력  무관 학력  고졸(예) 지역  서울 마감일  06/20(목) 입사지원하기 렙쇼메이 기업의 진행중인 채용정보 보기





 해피랜드 코퍼레이션  해피랜드(의류그래픽/용품디자인) 경력직 모집  회사명  해피랜드 코퍼레이션 모집내용  해피랜드 코퍼레이션  해피랜드(의류그래픽/용품디자인) 경력직 모집 직종   근무형태  정규직 경력  경력 학력  무관 지역  서울 마감일  채용시까지 입사지원하기 해피랜드 코퍼레이션 기업의 진행중인 채용정보 보기





 해피랜드 코퍼레이션  스릭슨 골프웨어 경력사원 모집  회사명  해피랜드 코퍼레이션 모집내용  해피랜드 코퍼레이션  스릭슨 골프웨어 경력사원 모집 직종   근무형태  정규직 경력  경력 학력  전졸(예) 지역  서울 마감일  채용시까지 입사지원하기 해피랜드 코퍼레이션 기업의 진행중인 채용정보 보기





 렙쇼메이  메종블랑쉬  영업MD 경력(계약직) 모집  회사명  렙쇼메이 모집내용  렙쇼메이  메종블랑쉬  영업MD 경력(계약직) 모집 직종   근무형태  계약직 경력  경력 학력  무관 지역  서울 마감일  06/20(목) 입사지원하기 렙쇼메이 기업의 진행중인 채용정보 보기





 삼성서울병원  삼성서울병원 간호본부 외래 계약직 간호사 채용  회사명  삼성서울병원 모집내용  삼성서울병원  삼성서울병원 간호본부 외래 계약직 간호사 채용 직종   근무형태  계약직 경력  무관 학력  무관 지역  서울 마감일  05/28(화) 입사지원하기 삼성서울병원 기업의 진행중인 채용정보 보기





 (주)KFC KOREA  KFC 정규직 매니저 신입/경력 모집  회사명  (주)KFC KOREA 모집내용  (주)KFC KOREA  KFC 정규직 매니저 신입/경력 모집 직종   근무형태  정규직 경력  무관 학력  무관 지역  서울 마감일  05/26(일) 입사지원하기 (주)KFC KOREA 기업의 진행중인 채용정보 보기


